In [3]:
%load_ext autoreload
%autoreload 2
import sys
# sys.path.append('/workspace')
from clearml import StorageManager, Dataset
# from config.default import TrainingConfig
# from config.list_optimizer import ListOptimizer
# conf = TrainingConfig()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from datetime import datetime

# Define the time of day thresholds for night/evening and day
night_threshold = datetime.strptime('18:00:00', '%H:%M:%S').time()
day_threshold = datetime.strptime('06:00:00', '%H:%M:%S').time()
print(night_threshold)
# Convert the date string to a datetime object
date_consignment = "Sat, 04 Mar 2023 18:50:34 GMT"
date_strf = datetime.strptime(date_consignment.replace(' GMT', ''), '%a, %d %b %Y %H:%M:%S')

# Extract the time component from the datetime object
time = date_strf.time()
# Categorize the date as night/evening or day based on the time of day
if time >= night_threshold or time < day_threshold:
    category = 'night/evening'
else:
    category = 'day'

print(category)

18:00:00
18:50:34
night/evening


In [34]:
type(vars(ListOptimizer))

mappingproxy

In [24]:
from dataclasses import asdict, dataclass, fields
asdict(conf)['hyp']

{'epoch': 10,
 'learning_rate': 0.0001,
 'opt_name': 'AdamW',
 'opt_weight_decay': 0,
 'opt_momentum': 0.9}

#### Manage Data

In [4]:
manager = StorageManager()
manager.download_folder(
    remote_url='app-data-workflow/dataset-dev/Bousteud/feedback-Bousteud-2023-02-26_keep-drop/keep-drop',
    local_folder='./dataset/',
    overwrite=True
)

ValueError: Target path "/home/agfian/shared/hdd_2/common-project/classifier/pytorch-lighting-image-classifier/src/notebooks/app-data-workflow/dataset-dev/Bousteud/feedback-Bousteud-2023-02-26_keep-drop/keep-drop" does not exist

In [ ]:
ds = Dataset.create(
    dataset_project='fruit-oil',
    dataset_name='sample-data-bousteud', 
    dataset_tags=['lerning-clearml'],
    output_uri=f'{conf.OUTPUT_URI}/dataset/sample-from-clearml'
)

ClearML results page: http://10.8.0.10:7080/projects/e3ab1f2c806947eea2ea59994d35ec50/experiments/411403ce990b433aaeac743d154d52a2/output/log
ClearML dataset page: http://10.8.0.10:7080/datasets/simple/e3ab1f2c806947eea2ea59994d35ec50/experiments/411403ce990b433aaeac743d154d52a2


In [6]:
ds.add_files(path='/workspace/dataset/simple')

Generating SHA2 hash for 420 files


100%|██████████| 420/420 [00:00<00:00, 4999.31it/s]

Hash generation completed


420

In [7]:
import os
root_folder = '/workspace/dataset/simple'
counts = []
folders = sorted(os.listdir(root_folder))
for folder in folders:
    count = len(os.listdir(os.path.join(root_folder, folder)))
    counts.append([count])


ds.get_logger().report_histogram(
    title='Dataset Histogram',
    series='Training Simple Dataset',
    values=counts,
    labels=folders,
    xaxis='class',
    yaxis='count of data'
)

In [8]:
ds.finalize(auto_upload=True)

Pending uploads, starting dataset upload to s3://10.8.0.66:9000/clearml-test/dataset/sample-from-clearml
Uploading dataset changes (420 files compressed to 69.21 MiB) to s3://10.8.0.66:9000/clearml-test/dataset/sample-from-clearml
File compression and upload completed: total size 69.21 MiB, 1 chunk(s) stored (average size 69.21 MiB)


True

In [10]:
ds.get_default_storage()

's3://10.8.0.66:9000/clearml-test/dataset/sample-from-clearml'

# Data Preparation

In [5]:
import glob
import torch
import albumentations as al
from os.path import join
from typing import Union
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import cv2
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import numpy as np



### Prepare Data

In [2]:
from random import shuffle

def get_list_data(root_path, conf:TrainingConfig):

    def check_health_img(fp):
        try:
            img = cv2.imread(fp)
            h,w,c = img.shape
            if h > 0 and w>0:
                return True
        except Exception as e:
            print('[ERROR] Image Corrupt: ', fp, e)
            return False

    def split_list(ls_fp_image):
        count_imgs = len(ls_fp_image)
        tr_count = int(tr*count_imgs)
        va_count = int(va*count_imgs)
        shuffle(ls_fp_image)
        train = ls_fp_image[:tr_count]
        val = ls_fp_image[tr_count:tr_count+va_count]
        test = ls_fp_image[tr_count+va_count:]
        return train, val, test
    
    d_metadata = {
        'ratio': [],
        'counts' : {
            'train': {},
            'val': {},
            'test': {},
        }
    }

    labels = conf.data.category
    d_data = {lbl:[] for lbl in labels}
    ls_train = []
    ls_val = []
    ls_test = []

    for label in labels:
        fp_folder = join(root_path, label)
        for file in os.listdir(fp_folder):
            fp_image = join(fp_folder, file)
            if check_health_img(fp_image):
                d_data[label].append((fp_image, labels.index(label)))
    
    tr = conf.data.train_ratio
    va = conf.data.val_ratio
    te = conf.data.test_ration
    
    d_metadata['ratio'] = [tr, va, te]

    ls_train_set, ls_val_set, ls_test_set = [], [], []
    for key, ls_fp_image in d_data.items():
        ls_train, ls_val, ls_test = split_list(ls_fp_image)
        ls_train_set.extend(ls_train)
        ls_val_set.extend(ls_val)
        ls_test_set.extend(ls_test)
        d_metadata['counts']['train'][key] = len(ls_train)
        d_metadata['counts']['val'][key] = len(ls_train)
        d_metadata['counts']['test'][key] = len(ls_train)

    d_metadata['train_count'] = len(ls_train_set)
    d_metadata['val_count'] = len(ls_val_set)
    d_metadata['test_count'] = len(ls_test_set)
    
    return ls_train_set, ls_val_set, ls_test_set, d_metadata

### Create Dataset

In [3]:
class ImageDatasetBinsho(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = al.Compose(transform)

    def __len__(self): return len(self.data)
    
    def __getitem__(self, index):
        fp_img, y  = self.data[index]
        y_label = torch.tensor(int(y))
        x_image = np.array(Image.open(fp_img)) # rgb format!
        x_image = self.transform(image=x_image)["image"] 
        return x_image, y_label


### Create DataLoader

In [141]:
def meanstd(dl):
    batch,sum_,sqr_= 0, 0, 0
    for x,y in dl:
        # print(type(x), x.shape, torch.min(x), torch.max(x))
        sum_+=torch.mean(x,axis=[0,2,3])
        sqr_+=torch.mean(x**2,axis=[0,2,3])
        batch+=1
    mean= sum_/batch
    std= (sqr_/batch)-mean**2
    print(mean,std)

In [142]:
imgs1 = ImageDatasetBinsho(root_path='/workspace/dataset/simple', conf=conf, transform=conf.aug.get_ls_val())
len(imgs1)

420

In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

# Note - you must have torchvision installed for this example
from torchvision import transforms


class ImageDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = "./"):
        super().__init__()
        self.data_dir = data_dir
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    
    def setup(self, stage: str):
        # get list of data
        self.conf = TrainingConfig()
        ls_train_set, ls_val_set, ls_test_set, d_metadata = get_list_data(root_path='/workspace/dataset/simple', conf=self.conf)
        
        # Assign train/val datasets for use in dataloaders
        if stage == "fit":
            self.data_train = ImageDatasetBinsho(ls_train_set, transform=self.conf.aug.get_ls_train())
            self.data_val = ImageDatasetBinsho(ls_val_set, transform=self.conf.aug.get_ls_train())

        # Assign test dataset for use in dataloader(s)
        if stage == "test":
            self.data_test = ImageDatasetBinsho(ls_test_set, transform=self.conf.aug.get_ls_train())

    def train_dataloader(self):

        return DataLoader(self.data_test, batch_size=self.conf.data.batch)

    def val_dataloader(self):
        return DataLoader(self.data_val, batch_size=self.conf.data.batch)

    def test_dataloader(self):
        return DataLoader(self.data_test, batch_size=self.conf.data.batch)

In [15]:
from clearml import StorageManager

# link = 's3://10.8.0.66:9000/app-data-workflow/dataset-dev/IOI/feedback-IOI-2023-03-02_keep-drop/keep-drop'
link = 's3://10.8.0.66:9000/app-data-workflow/dataset-dev/IOI/feedback-IOI-2023-03-02_keep-drop/keep-drop'
ls_files = StorageManager.list(
        remote_url=link,
        return_full_path=True,
        with_metadata=True
    )
print('Total Data:', len(ls_files))

Total Data: 0


In [17]:
from minio import Minio

endpoint="10.8.0.66:9000"
access_key="bs_server_1"
secret_key="zNAYleEDeCnlzaXJsd7MvXnQhPmZehIA"
bucket_name="app-data-workflow"
region = 'binsho-server-2'
# dataset = dataset
# download_dir = download_dir

# Create a Minio client with the given credentials
minio_client = Minio(
    endpoint,
    access_key=access_key,
    secret_key=secret_key,
    secure=False,
    region=region
)

In [67]:
l = 'dataset-dev/IOI/feedback-ioi-2023-03-02_keep-drop/keep-drop'
# l = 'dataset-dev/IOI/feedback-IOI-2023-03-02_keep-drop/keep-drop'
the_data = minio_client.list_objects(bucket_name=bucket_name, prefix=l, recursive=True)

In [68]:
for d in the_data:
    print(d.object_name)

dataset-dev/IOI/feedback-ioi-2023-03-02_keep-drop/keep-drop/drop/64016b3d9b8e29c974cd580f_Ripe_ffb_11.png
dataset-dev/IOI/feedback-ioi-2023-03-02_keep-drop/keep-drop/drop/64016b3d9b8e29c974cd580f_Ripe_ffb_15.png
dataset-dev/IOI/feedback-ioi-2023-03-02_keep-drop/keep-drop/drop/64016b3d9b8e29c974cd580f_Ripe_ffb_4.png
dataset-dev/IOI/feedback-ioi-2023-03-02_keep-drop/keep-drop/drop/64016b3d9b8e29c974cd580f_Underripe_ffb_19.png
dataset-dev/IOI/feedback-ioi-2023-03-02_keep-drop/keep-drop/drop/64016b3d9b8e29c974cd580f_Underripe_ffb_20.png
dataset-dev/IOI/feedback-ioi-2023-03-02_keep-drop/keep-drop/drop/64016b3d9b8e29c974cd580f_Underripe_ffb_27.png
dataset-dev/IOI/feedback-ioi-2023-03-02_keep-drop/keep-drop/drop/64016b3d9b8e29c974cd580f_Underripe_ffb_3.png
dataset-dev/IOI/feedback-ioi-2023-03-02_keep-drop/keep-drop/drop/64016b3d9b8e29c974cd580f_Underripe_ffb_40.png
dataset-dev/IOI/feedback-ioi-2023-03-02_keep-drop/keep-drop/drop/64016b3d9b8e29c974cd580f_Underripe_ffb_41.png
dataset-dev/IOI/fe

In [72]:
import plotly.graph_objs as go

# Sample data
x_values = ['Empty', 'Class 2', 'Class 3', 'Class 4']
y_train = [50, 30, 70, 90]
y_test = [20, 40, 60, 80]
y_val = [10, 20, 30, 40]

# Create three bar chart traces, one for each section
trace1 = go.Bar(x=x_values, y=y_train, name='Train', text=y_train, textposition='auto', marker=dict(color='#1f77b4'))
trace2 = go.Bar(x=x_values, y=y_test, name='Test', text=y_test, textposition='auto', marker=dict(color='#ff7f0e'))
trace3 = go.Bar(x=x_values, y=y_val, name='Validation', text=y_val, textposition='auto', marker=dict(color='#2ca02c'))

# Create a layout for the chart
layout = go.Layout(title='Data Distribution', xaxis=dict(title='Class Name', showticklabels=True), yaxis=dict(title='Counts', showticklabels=True), barmode='group', legend=dict(x=0, y=1))

# Create a figure object that contains the traces and layout
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)

# Update the font and background colors of the chart
fig.update_layout(font=dict(color='white'), plot_bgcolor='#2c3e50', paper_bgcolor='#2c3e50')

# Show the chart
fig.show()


In [74]:
import random
from datetime import datetime

# Define a list of supplier names and a list of maturity levels
supplier_names = ['Supplier A', 'Supplier B', 'Supplier C']
maturity_levels = ['Early', 'Mid', 'Late']

# Create a dummy d_metadata dictionary with 100 entries
d_metadata = []
for i in range(100):
    c = {
        "date_consignment": datetime.now(),
        "penalty": random.uniform(0, 1),
        "suppliername": random.choice(supplier_names),
        "ticketnumber": f'Ticket-{i}',
        "uid_consignment": f'UID-{i}',
        "weight": round(random.uniform(1, 100), 3),
        "maturity": random.choice(maturity_levels),
        "mills": f'Mill-{i}'
    }
    d_metadata.append(c)


In [76]:
import pandas as pd
import plotly.express as px

# Create a DataFrame from the d_metadata dictionary
df = pd.DataFrame.from_dict(d_metadata)

# Group the DataFrame by suppliername and maturity and count the number of entries in each group
grouped_df = df.groupby(['suppliername', 'maturity']).size().reset_index(name='count')

# Create a bar plot using Plotly
fig = px.bar(grouped_df, x='suppliername', y='count', color='maturity', barmode='group', text_auto=True)

# Show the plot
fig.show()


In [80]:
import requests
from requests_toolbelt import MultipartEncoder
# enter pip install requests_toolbelt to install dependent libraries
def uploadImage():
    url = "https://open.larksuite.com/open-apis/bot/v2/hook/e8838178-2b69-49c0-b24e-006901b4b650"
    form = {'image_type': 'message',
            'image': (open('newplot.png', 'rb'))}  # 需要替换具体的path 
    multi_form = MultipartEncoder(form)
    # headers = {
    #     'Authorization': 'Bearer t-xxx',  ## 获取tenant_access_token, 需要替换为实际的token
    # }
    # headers['Content-Type'] = multi_form.content_type
    response = requests.request("POST", url,  data=multi_form)
    print(response.headers['X-Tt-Logid'])  # for debug or oncall
    print(response.content)  # Print Response

uploadImage()

20230314121340BDF50DD1545CC60F72AA
b'{"code":9499,"msg":"Bad Request","data":{}}'


In [88]:
from collections import Counter
from prettytable import PrettyTable

my_dict = {'apple': 3, 'banana': 2, 'orange': 5, 'kiwi': 1}

# Convert the dictionary to a list of tuples sorted by descending order of counts
my_list = sorted(my_dict.items(), key=lambda x: x[1], reverse=True)

# Create a pretty table with two columns: item and count
table = PrettyTable(['Item', 'Count'])

# Add the rows to the table
for item, count in my_list:
    table.add_row([item, count])

# Print the table to the terminal
print(table)


+--------+-------+
|  Item  | Count |
+--------+-------+
| orange |   5   |
| apple  |   3   |
| banana |   2   |
|  kiwi  |   1   |
+--------+-------+


In [91]:
from collections import Counter
from tabulate import tabulate

my_dict = {'apple': 3, 'banana': 2, 'orange': 5, 'kiwi': 1}

# Convert the dictionary to a list of tuples sorted by descending order of counts
my_list = sorted(my_dict.items(), key=lambda x: x[1], reverse=True)

# Create a list of lists representing the table data
table_data = [['Item', 'Count']]
for item, count in my_list:
    table_data.append([item, count])

# Format the table as Markdown
table_md = tabulate(table_data, tablefmt='pipe')

# Print the table to the console
print(table_md)


|:-------|:------|
| Item   | Count |
| orange | 5     |
| apple  | 3     |
| banana | 2     |
| kiwi   | 1     |


In [113]:
import math

my_data = [125, 423, 227, 322, 51, 1023]
key = ['asd', 'MEte', 'asddas', 'asdsaddas', 'asda', 'asdf']

# Calculate the maximum value in the data
max_value = max(my_data)

# Calculate the logarithmic scale factor for the diagram
scale_factor = 25 / math.log(max_value)

# Calculate the maximum label length
max_label_len = max(len(label) for label in key)

# Create a list of strings representing the diagram
diagram = []
for idx, value in enumerate(my_data):
    label = key[idx].ljust(max_label_len)
    bar_length = int(math.log(value) * scale_factor)
    bar = "|" * bar_length
    diagram.append(f"{label} {value:>4} |{bar}")
    
# Create a header for the diagram
header = " " * (max_label_len + 1) + "Count"
line = "-" * len(header)
diagram.insert(0, header)
diagram.insert(1, line)

# Join the diagram strings into a single multi-line string
diagram_str = "\n".join(diagram)

# Print the diagram to the console
print(diagram_str)


          Count
---------------
asd        125 ||||||||||||||||||
MEte       423 ||||||||||||||||||||||
asddas     227 ||||||||||||||||||||
asdsaddas  322 |||||||||||||||||||||
asda        51 |||||||||||||||
asdf      1023 ||||||||||||||||||||||||||


In [115]:
import math

my_data = [125, 423, 227, 322, 51, 1023]
key = ['asd', 'MEte', 'asddas', 'asdsaddas', 'asda', 'asdf']

# Calculate the maximum value in the data
max_value = max(my_data)

# Calculate the logarithmic scale factor for the diagram
scale_factor = 25 / math.log(max_value)

# Create a list of strings representing the diagram
diagram = []
for idx, value in enumerate(my_data):
    label = key[idx][:4].ljust(4)
    bar_length = int(math.log(value) * scale_factor)
    bar = "█" * bar_length
    diagram.append(f"{label} {bar}")
    
# Join the diagram strings into a single multi-line string
diagram_str = "\n".join(diagram)

# Print the diagram to the console
print(diagram_str)


asd  █████████████████
MEte █████████████████████
asdd ███████████████████
asds ████████████████████
asda ██████████████
asdf █████████████████████████


In [111]:
import math

my_data = [125, 423, 227, 322, 51, 1023]

# Calculate the maximum value in the data
max_value = max(my_data)

# Calculate the logarithmic scale factor for the diagram
scale_factor = 25 / math.log(max_value)

# Create a list of strings representing the diagram
diagram = []
for value in my_data:
    bar_length = int(math.log(value) * scale_factor)
    bar = "|" * bar_length
    diagram.append(f"{value:>4} {bar[::-1]}")

# Reverse the order of the strings and join them into a single multi-line string
diagram_str = "\n".join(reversed(diagram))

# Print the diagram to the console
print(diagram_str)


1023 |||||||||||||||||||||||||
  51 ||||||||||||||
 322 ||||||||||||||||||||
 227 |||||||||||||||||||
 423 |||||||||||||||||||||
 125 |||||||||||||||||


In [116]:
requests.post(
    url="https://open.larksuite.com/open-apis/bot/v2/hook/e8838178-2b69-49c0-b24e-006901b4b650",
    json={
    "msg_type": "text",
    "content": {
        "text":  f"""
{diagram_str}
        """
            }
        } 
    )

<Response [200]>

In [82]:
requests.post(
    url="https://open.larksuite.com/open-apis/bot/v2/hook/e8838178-2b69-49c0-b24e-006901b4b650",
    json={
    "msg_type": "image",
    "content": {
                "image": open('newplot.png', 'rb')
            }
        } 
    )

TypeError: Object of type BufferedReader is not JSON serializable